# Exercise 07

## Kaggle competition 

* Overview of how Kaggle works ([slides](https://github.com/justmarkham/DAT8/raw/master/slides/16_kaggle.pdf))
* Kaggle Titanic competition: [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)

# Exercise 07.1 (20 points)

Create a submission using different classification methods, preprocessing extrategies and cross-validation techniques discussed during the class. The output must be detailed in this notebook.

# Exercise 07.2 (20 points)

The reminder points will be allocated based on the performance of each one submission.

# Mejor Modelo

In [135]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('titanic_train.csv', header=0)
test_df = pd.read_csv('titanic_test.csv', header=0)

In [136]:
titanic = train_df
titanic.Age.fillna(titanic.Age.mean(), inplace=True)
titanic.Embarked.fillna('S', inplace=True)
titanic['Sex_Female'] = titanic.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
titanic = pd.concat([titanic, embarked_dummies], axis=1)
titanic['Age2'] = titanic['Age'] ** 2
titanic['SibSp2'] = titanic['SibSp'] ** 2
titanic['Parch2'] = titanic['Parch'] ** 2
titanic['Fare2'] = titanic['Fare'] ** 2

In [137]:
# define X and y
feature_cols = ['Pclass', 'Age', 'SibSp','Parch','Fare','Sex_Female', 'Embarked_C', 
                'Embarked_S', 'Age2', 'SibSp2','Fare2']
X = titanic[feature_cols]
y = titanic.Survived

# Create k-folds
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
kf = KFold(X.shape[0], n_folds=10, random_state=0)

results = []
models = []
for train_index, test_index in kf:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # train a logistic regression model
    models.append(LogisticRegression(C=1e9))
    
    models[-1].fit(X_train, y_train)

    # make predictions for testing set
    y_pred_class = models[-1].predict(X_test)

    # calculate testing accuracy
    results.append(metrics.accuracy_score(y_test, y_pred_class))
results

[0.76666666666666672,
 0.8314606741573034,
 0.7865168539325843,
 0.8539325842696629,
 0.7752808988764045,
 0.7865168539325843,
 0.7865168539325843,
 0.7752808988764045,
 0.8651685393258427,
 0.8089887640449438]

In [138]:
test_df.Age.fillna(titanic.Age.mean(), inplace=True)
test_df.Fare.fillna(titanic.Fare.mean(), inplace=True) 
test_df.Embarked.fillna('S', inplace=True)
test_df['Sex_Female'] = test_df.Sex.map({'male':0, 'female':1})
embarked_dummies = pd.get_dummies(test_df.Embarked, prefix='Embarked')
test_df = pd.concat([test_df, embarked_dummies], axis=1)
test_df['Age2'] = test_df['Age'] ** 2
test_df['SibSp2'] = test_df['SibSp'] ** 2
test_df['Parch2'] = test_df['Parch'] ** 2
test_df['Fare2'] = test_df['Fare'] ** 2

In [139]:
probas = pd.DataFrame(index=test_df[feature_cols].index, columns=['p'+ str(i) for i in range(10)])

In [140]:
for i in range(10):
    proba = models[i].predict_proba(test_df[feature_cols])[:, 1]
    probas.iloc[:, i] = proba

In [134]:
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission_52.csv", index=False)